In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil import parser

final_merged_data = pd.read_csv("final_merged_data.csv")

In [2]:
final_merged_data.columns

Index(['date', 'net_sentiment', 'stock', 'sofr_Rate Type', 'sofr_Rate (%)',
       'sofr_1st Percentile (%)', 'sofr_25th Percentile (%)',
       'sofr_75th Percentile (%)', 'sofr_99th Percentile (%)',
       'sofr_Volume ($Billions)', 'sofr_Target Rate From (%)',
       'sofr_Target Rate To (%)', 'sofr_Intra Day - Low (%)',
       'sofr_Intra Day - High (%)', 'sofr_Standard Deviation (%)',
       'sofr_30-Day Average SOFR', 'sofr_90-Day Average SOFR',
       'sofr_180-Day Average SOFR', 'sofr_SOFR Index',
       'sofr_Revision Indicator (Y/N)', 'sofr_Footnote ID', 'unemp_UNEMPLOY',
       'vix_VIXCLS', 'fomc_month', 'fomc_raw_date_text', 'fomc_year',
       'aapl_Open', 'aapl_High', 'aapl_Low', 'aapl_Close', 'aapl_Adj Close',
       'aapl_Volume', 'amzn_Open', 'amzn_High', 'amzn_Low', 'amzn_Close',
       'amzn_Adj Close', 'amzn_Volume', 'fed_DFEDTARU', 'fed_rate_change',
       'fed_action'],
      dtype='object')

In [5]:
df = final_merged_data.copy()

df['date'] = pd.to_datetime(df['date'], errors='coerce')


for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].astype(str).str.replace(",", "").str.replace("%", "")
        df[col] = pd.to_numeric(df[col], errors="ignore")

for col in df.columns:
    if df[col].dtype == "object":
        sample = df[col].dropna().head(20)
        if all(s.replace('.', '', 1).isdigit() for s in sample if isinstance(s, str)):
            df[col] = pd.to_numeric(df[col], errors="coerce")

print(df.dtypes)


date                             datetime64[ns]
net_sentiment                           float64
stock                                    object
sofr_Rate Type                           object
sofr_Rate (%)                           float64
sofr_1st Percentile (%)                 float64
sofr_25th Percentile (%)                float64
sofr_75th Percentile (%)                float64
sofr_99th Percentile (%)                float64
sofr_Volume ($Billions)                 float64
sofr_Target Rate From (%)               float64
sofr_Target Rate To (%)                 float64
sofr_Intra Day - Low (%)                float64
sofr_Intra Day - High (%)               float64
sofr_Standard Deviation (%)             float64
sofr_30-Day Average SOFR                float64
sofr_90-Day Average SOFR                float64
sofr_180-Day Average SOFR               float64
sofr_SOFR Index                         float64
sofr_Revision Indicator (Y/N)           float64
sofr_Footnote ID                        

C:\Users\Huawei\AppData\Local\Temp\ipykernel_47488\2682829487.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\Huawei\AppData\Local\Temp\ipykernel_47488\2682829487.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\Huawei\AppData\Local\Temp\ipykernel_47488\2682829487.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\Huawei\AppData\Local\Temp\ipykernel_47488\2682829487.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_n

In [6]:
numeric_fix_cols = ["aapl_Open", "aapl_Volume", "amzn_Volume"]

for col in numeric_fix_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "")
        .str.replace("%", "")
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

print(df[numeric_fix_cols].dtypes)


aapl_Open      float64
aapl_Volume    float64
amzn_Volume    float64
dtype: object


In [7]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

df = final_merged_data.copy()

# rename the control variables for convenience
df.rename(columns={
    'vix_VIXCLS': 'vix',
    'unemp_UNEMPLOY': 'unemp',
    'fed_DFEDTARU': 'fed_rate'
}, inplace=True)

# choose relevant columns
df['Volume'] = df.apply(
    lambda r: r['aapl_Volume'] if r['stock']=='AAPL' else r['amzn_Volume'],
    axis=1
)

# delete rows with NA volume
df = df[df['Volume'].notna()].copy()

# dependent variable log_volume
df['log_volume'] = np.log(df['Volume'] + 1)

# independent variable exposure
df['exposure'] = df['net_sentiment']

# time processing
df['date'] = pd.to_datetime(df['date'])
df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')
df = df.sort_values(['stock','date'])

# lagged variables
df['exposure_lag1'] = df.groupby('stock')['exposure'].shift(1)
df['log_volume_lag1'] = df.groupby('stock')['log_volume'].shift(1)

# drop NA rows
df = df.dropna(subset=['log_volume','exposure'])

# =============== IV ===============
def run_ols(formula, data, cluster=None):
    m = smf.ols(formula=formula, data=data).fit()
    if cluster is not None:
        m = m.get_robustcov_results(cov_type='cluster', groups=data[cluster])
    else:
        m = m.get_robustcov_results(cov_type='HC1')
    print(m.summary())
    return m


# =============== SINGLE STOCK ANALYSIS ===============

def analyze_stock(stock):
    print(f"\n================ 分析 {stock} ================\n")
    sub = df[df['stock']==stock].dropna(subset=['log_volume','exposure'])

    # baseline OLS
    print(">> Baseline OLS:")
    run_ols(
        "log_volume ~ exposure + vix + unemp + fed_rate",
        sub
    )

    # add lagged vars
    print("\n>> OLS + lagged vars:")
    run_ols(
        "log_volume ~ exposure + exposure_lag1 + log_volume_lag1 + vix + unemp + fed_rate",
        sub
    )

    #fixed effects
    print("\n>> Time FE:")
    run_ols(
        "log_volume ~ exposure + exposure_lag1 + log_volume_lag1 + vix + unemp + fed_rate + C(date_str)",
        sub
    )

analyze_stock("AAPL")
analyze_stock("AMZN")

# =============== mergerd pooled analysis ===============

print("\n================ Merged Pooled Analysis ================\n")

run_ols(
    """
    log_volume ~ exposure + exposure_lag1 + log_volume_lag1
                 + vix + unemp + fed_rate
                 + C(stock) + C(date_str)
    """,
    df,
    cluster="stock"
)

# add interaction term for AMZN
df["is_amzn"] = (df["stock"]=="AMZN").astype(int)

run_ols(
    """
    log_volume ~ exposure + exposure:is_amzn
                 + exposure_lag1 + log_volume_lag1
                 + vix + unemp + fed_rate
                 + C(stock) + C(date_str)
    """,
    df,
    cluster="stock"
)


# =============== 5. DiD===============

def run_did(treat_stock, event_date):
    print(f"\n=============== DiD: treat = {treat_stock}, event_date = {event_date} ===============\n")

    dd = df.copy()
    event_date = pd.to_datetime(event_date)

    dd["treat"] = (dd["stock"] == treat_stock).astype(int)
    dd["post"] = (dd["date"] >= event_date).astype(int)
    dd["did"] = dd["treat"] * dd["post"]

    run_ols(
        """
        log_volume ~ treat + post + did
                     + exposure + exposure_lag1 + log_volume_lag1
                     + vix + unemp + fed_rate
                     + C(stock) + C(date_str)
        """,
        dd,
        cluster="stock"
    )

# change the event date as needed
event = "2020-03-01"

run_did("AAPL", event)
run_did("AMZN", event)

TypeError: can only concatenate str (not "int") to str

In [8]:
print(df.select_dtypes(include="object").columns)

Index(['date', 'stock', 'sofr_Rate Type', 'fomc_month', 'fomc_raw_date_text',
       'aapl_Open', 'aapl_Volume', 'amzn_Volume', 'fed_action', 'Volume'],
      dtype='object')


In [9]:
import pandas as pd

df = final_merged_data.copy()

numeric_object_cols = ["aapl_Open", "aapl_Volume", "amzn_Volume", "Volume"]

for col in numeric_object_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.replace(" ", "", regex=False)
        .str.replace("-", "", regex=False)
        .replace("", None)
    )

    df[col] = pd.to_numeric(df[col], errors="coerce")

print(df[numeric_object_cols].dtypes)
print(df[numeric_object_cols].head())


KeyError: 'Volume'